# Примеры моих запросов PostgreSQL

Ниже приведены несколько запросов. написанных мной, в рамках учебной программы.

## Задача 1

Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
inv_2011 AS (SELECT country_code,
                    AVG(funding_total) AS avg_2011
             FROM company
             WHERE EXTRACT(YEAR FROM fOunded_at) = 2011
             GROUP BY country_code),

inv_2012 AS (SELECT country_code,
                    AVG(funding_total)  AS avg_2012
             FROM company
             WHERE EXTRACT(YEAR FROM fOunded_at) = 2012
             GROUP BY country_code),

inv_2013 AS (SELECT country_code,
                    AVG(funding_total)  AS avg_2013
             FROM company
             WHERE EXTRACT(YEAR FROM fOunded_at) = 2013
             GROUP BY country_code)             

SELECT inv_2011.country_code,
       inv_2011.avg_2011,
       inv_2012.avg_2012,
       inv_2013.avg_2013
 
FROM  inv_2011
INNER JOIN inv_2012 ON inv_2011.country_code = inv_2012.country_code
INNER JOIN inv_2013 ON inv_2011.country_code = inv_2013.country_code
ORDER BY inv_2011.avg_2011 DESC

Результат запроса (первые 10 строк)

|country_code|avg_2011|avg_2012|avg_2013|
|---|---|---|---|
|PER|4000000|41000|25000|
|USA|2.24396e+06|1.20671e+06|1.09336e+06|
|HKG|2.18078e+06|226227|0|
|PHL|1.75e+06|4218.75|2500|
|ARE|1.718e+06|197222|35333.3|
|JPN|1.66431e+06|674720|50000|
|AUT|1.5342e+06|147806|85773.3|
|BRA|1.38007e+06|240639|67944.4|
|DEU|1.1288e+06|1.32915e+06|66612.7|
|ISR|1.03076e+06|1.27121e+06|294022|

## Задача 2

Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:

    номер месяца, в котором проходили раунды;
    количество уникальных названий фондов из США, которые инвестировали в этом месяце;
    количество компаний, купленных за этот месяц;
    общая сумма сделок по покупкам в этом месяце.

In [ ]:
SELECT EXTRACT(MONTH FROM fr.funded_at) AS monthh,
    COUNT(DISTINCT f.name) AS fund_name,
    COUNT(a.acquired_company_id) AS purchases,
    SUM(a.price_amount) AS sum_purchases
    
FROM funding_round AS fr
INNER JOIN investment AS i ON fr.id = i.funding_round_id
INNER JOIN fund AS f ON i.fund_id = f.id
RIGHT OUTER JOIN company AS c ON i.company_id = c.id
FULL OUTER JOIN acquisition AS a ON c.id = a.acquired_company_id

WHERE EXTRACT(YEAR FROM fr.funded_at) BETWEEN 2010 AND 2013
    AND f.country_code = 'USA'

GROUP BY monthh

Результат запроса

|monthh|fund_name|purchases|sum_purchases|
|---|---|---|---|
|1|815|204|1.61211e+10|
|2|637|129|4.04339e+10|
|3|695|179|1.12549e+10|
|4|718|187|4.11024e+10|
|5|695|130|9.2698e+09|
|6|785|205|2.10111e+10|
|7|803|119|1.4561e+10|
|8|726|170|1.03931e+10|
|9|793|153|2.6312e+10|
|10|764|156|2.4613e+10|
|11|661|142|2.3009e+10|
|12|590|90|6.5475e+09|


## Задача 3

На сколько процентов ежегодно менялось количество заказов в Northwind с 1996 по 1998 годы. Отобразите таблицу со следующими полями:

    Число года.
    Количество заказов за год.
    Процент, округлённый до целого числа, который показывает, насколько изменилось количество заказов в текущем году по сравнению с предыдущим. Для 1996 года выведите значение NULL.

In [ ]:
SELECT *,
ROUND(CAST(orders_count - LAG(orders_count) OVER (ORDER BY creation_year) AS numeric) / LAG(orders_count, 1, 0) OVER (ORDER BY creation_year) * 100)
FROM (
SELECT EXTRACT(YEAR FROM order_date) AS creation_year, 
    COUNT(order_id) AS orders_count
FROM northwind.orders
WHERE EXTRACT(YEAR FROM order_date) BETWEEN 1996 AND 1998
GROUP BY creation_year
ORDER BY EXTRACT(YEAR FROM order_date)
    ) AS ord_num

|creation_year|orders_count|round|
|---|---|---|
|1996|152|[NULL]|
|1997|408|168|
|1998|270|-34|